# 🧠 Feature Engineering Notebook. 

**Objective:** 

This notebook focuses on creating and transforming features that can improve model performance. We’ll derive `new features`, `encode categorical` variables, and `prepare the final dataset` for `modeling`.

In [4]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Load preprocessed dataset
processed_data_path = "../data/processed_student_data.csv"
df = pd.read_csv(processed_data_path)

print(f"✅ Data loaded successfully! Shape: {df.shape}")
df.head()

✅ Data loaded successfully! Shape: (2392, 15)


,studentid,age,gender,ethnicity,parentaleducation,studytimeweekly,absences,tutoring,parentalsupport,extracurricular,sports,music,volunteering,gpa,gradeclass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0


In [ ]:
##### Feature Engineering #####

# feature_engineered_data_path = "feature_engineered_student_data.csv"
# df.to_csv(feature_engineered_data_path, index=False)

# print(f"✅ Feature-engineered data saved: {feature_engineered_data_path}")
